In [15]:
import requests 
import pandas as pd 
from sqlalchemy import create_engine


This program enables users to analyze flight data retrieved from the [AviationStack API](https://aviationstack.com/documentation). It allows users to track flight details departing from any airport over a selected date or date range. The retrieved data is then stored locally in a SQL database for further analysis.

Important Notes Before You Begin:

* This program is designed for users subscribed to the Basic Plan of the AviationStack API.
* You must register and obtain your personal access_key from the [AviationStack pricing page](https://aviationstack.com/pricing).
* AviationStack API limits date range queries to only 3 months from the current date. 


In [16]:
def get_paginated_data(endpoint: str, access_key: str, extra_params: dict = None, limit: int = 100):

    """
    Generalized pagination handler for AviationStack API.

    Parameters:
        endpoint (str): The API endpoint (e.g. 'flights', 'cities').
        access_key (str): Your API key.
        extra_params (dict): Additional query parameters (e.g., {'dep_iata': 'SFO', 'flight_date': '2025-06-01'})
        limit (int): Max records per page (default is 100 for free/basic plans).

    Returns:
        A list of all paginated records from the endpoint.
    """

    import time
    import math

    base_url = f"https://api.aviationstack.com/v1/{endpoint}"
    offset = 0
    all_data = []

    # Initialize parameters
    params = {
        "access_key": access_key,
        "limit": limit,
        "offset": offset
    }

    if extra_params:
        params.update(extra_params)

    # Initial API call to extract total record count from pagination info
    r = requests.get(base_url, params=params)
    r.raise_for_status()
    data = r.json()

    # Get total records 
    total_records = data.get("pagination", {}).get("total", 0)
    all_data.extend(data.get("data", []))

    # If the total number of records is below the limit, the response will contain all records on the first page.
    if total_records <= limit:
        return all_data

    # Calculate the total number of pages needed to retrieve all data
    num_pages = math.ceil(total_records / limit)

    for i in range(1, num_pages):
        offset = i * limit
        params["offset"] = offset
        time.sleep(1) # Wait 1 second between each request
        r = requests.get(base_url, params=params)
        r.raise_for_status()
        data = r.json()
        all_data.extend(data.get("data", []))

    return all_data


In [17]:
def get_flights_in_range(access_key: str, start_date, end_date, dep_code_list: list = None, arr_code_list: list = None, 
                         airline_code_list: list = None, limit: int = 100):
    """
    Retrieves all flight data for a given date or date range, filtered by departure and/or arrival airports and optionally by airline.

    Parameters:
        access_key (str): Registered API key from AviationStack
        start_date (str): Start of date range (format: 'YYYY-MM-DD')
        end_date (str): End of date range (format: 'YYYY-MM-DD')
        dep_code_list (list, optional): List of departure IATA codes (e.g., ['SFO'])
        arr_code_list (list, optional): List of arrival IATA codes (e.g., ['JFK'])
        airline_code_list (list, optional): List of airline IATA codes (e.g., ['DL', 'UA'] for Delta & United)
        limit (int): Max records per page (default 100)

    Returns:
        List of flight records or None if error occurs
    """
    from datetime import datetime, timedelta

    if not (dep_code_list or arr_code_list or airline_code_list):
        raise ValueError("You must provide at least one of dep_code_list, arr_code_list, or airline_code_list.")

    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end_dt - start_dt

    all_responses = []

    try:
        for i in range(delta.days + 1):
            current_date = (start_dt + timedelta(days=i)).strftime("%Y-%m-%d")

            dep_codes = dep_code_list or [None]
            arr_codes = arr_code_list or [None]
            airline_codes = airline_code_list or [None]

            for dep_code in dep_codes:
                for arr_code in arr_codes:
                    for airline_code in airline_codes:
                        params = {"flight_date": current_date}
                        if dep_code:
                            params["dep_iata"] = dep_code
                        if arr_code:
                            params["arr_iata"] = arr_code
                        if airline_code:
                            params["airline_iata"] = airline_code

                        flights = get_paginated_data("flights", access_key, params, limit)
                        all_responses.extend(flights)

        return all_responses

    except requests.exceptions.HTTPError as http_error:
        print(f"An HTTP error occurred: {http_error}")
    except requests.exceptions.RequestException as req_error:
        print(f"Request failed: {req_error}")

    return None


In [18]:
def parse_flight_response(response: list):
    """ 
    Convert the flights json response into a list of dictionaries 
    Parameters:
        response (list): The list of dictionary returned from calling `get_all_flights'

    Returns:
        list: A new list of dictionaries containing selected flight details.
    """

    flight_list = [] 

    for flight in response:
        flight_details = {
            "flight_date": flight["flight_date"], # date of flight
            "flight_number": flight["flight"]["number"], # flight number
            "flight_status": flight["flight_status"], # 'scheduled', 'active', 'landed', 'cancelled', 'incident', 'diverted'
            "dep_airport": flight["departure"]["airport"], # name of departure aiport
            "dep_timezone": flight["departure"]["timezone"], # departure timezone
            "dep_iata": flight["departure"]["iata"], # iata code for departure location/airport
            "dep_delay": flight["departure"]["delay"], # departure delay time in minutes
            "scheduled_departure_datetime": flight["departure"]["scheduled"], # scheduled departure date and time 
            "actual_departure_datetime": flight["departure"]["actual"], # actual departure date and time 
            "arr_airport": flight["arrival"]["airport"], # name of arrival airport 
            "arr_timezone": flight["arrival"]["timezone"], # arrival timezone
            "arr_iata": flight["arrival"]["iata"], # iata code for the arrival locatiion/airport
            "arr_delay": flight["arrival"]["delay"], # arrival delay in minutes 
            "scheduled_arrival_datetime": flight["arrival"]["scheduled"], # scheduled arrival date and time 
            "actual_arrival_datetime": flight["arrival"]["actual"],
            "airline": flight["airline"]["name"], # name of airline 
        }
    
        flight_list.append(flight_details)           

    return flight_list


In [19]:
def merge_cities(flights_df, airports_csv, iata_column, column_prefix):
    
    """ 
    Merge cities dataframe with flights dataframe
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data 
        airports_csv (str): csv file containing airport location details borrowed from https://github.com/lxndrblz/Airports/blob/main/airports.csv
        iata_column (str): Column name in flights_df that contains the airport IATA codes to join on (e.g., "dep_iata" or "arr_iata").
        column_prefix (str): Prefix to add to merged airport columns for clarity (e.g., "dep_" for departure-related columns).

    Returns:
        pd.DataFrame: A new DataFrame with airport location information merged into flights_df.
    """

    flights_copy = flights_df.copy()
    airports_df = pd.read_csv(airports_csv)

    # Extract key columns from dataframe
    sub_airports_df = airports_df[["code", "latitude", "longitude", "country", "city", "state"]].copy()
    
    # Update the column names to include prefix (e.g. dep_) for clarity
    airports_colnames = sub_airports_df.columns
    airport_map = {col: column_prefix + col for col in airports_colnames}
    sub_airports_df.rename(columns = airport_map, inplace=True)
    
    merge_cities = pd.merge(flights_copy, sub_airports_df, how="left", left_on = iata_column, right_on = column_prefix + "code") 
    merge_cities.drop(columns=column_prefix + "code", inplace=True)

    return merge_cities
  


In [20]:
def drop_duplicates(flights_df: pd.DataFrame):
    
    """ 
    Drop duplicates rows
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data 
    Return:
        Deduplicated dataframe
    """
    deduped_df = flights_df.drop_duplicates()
    
    return deduped_df

In [21]:
def parse_datetime(flights_df: pd.DataFrame, datetime_column: str):
    
    """
    Parses out the datetime column into a date and time column
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data 
        datetime_column (str): name of datetime column to parse
    Return:
        flights_df with date and time parsed into different columns
    """

    # Convert the datetime column to pandas datetime
    dt_object = pd.to_datetime(flights_df[datetime_column], errors='coerce')

    # Extract date and time components and add as new columns
    flights_df[f"{datetime_column}_date"] = dt_object.dt.date
    flights_df[f"{datetime_column}_time"] = dt_object.dt.time

    return flights_df

In [22]:
def change_status(flights_df: pd.DataFrame):

    """
    Update active and scheduled flight statuses
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data
    Returns:
        DataFrame with updated flight statuses
    """
    flights_copy = flights_df.copy()
    
    # Setup datetime calculations
    flights_copy["flight_date"] = pd.to_datetime(flights_copy["flight_date"], utc=True, errors="coerce")
    today = pd.Timestamp.now(tz="UTC").normalize()
    is_historical = (today - flights_copy["flight_date"]).dt.days > 2 # Flights > 2 days old is considered "historical"
    
    # Define status transition rules: (current_status, condition, new_status)
    status_rules = [
        # Scheduled flights
        ("scheduled", flights_copy["actual_departure_datetime"].notna(), "active"),
        ("scheduled", is_historical & flights_copy["actual_departure_datetime"].isna(), "pending update"),
        
        # Active flights  
        ("active", flights_copy["actual_arrival_datetime"].notna(), "landed"),
        ("active", is_historical & flights_copy["actual_arrival_datetime"].isna(), "pending update"),
    ]
    
    # Apply status rules
    for current_status, condition, new_status in status_rules:
        mask = (flights_copy["flight_status"] == current_status) & condition
        flights_copy.loc[mask, "flight_status"] = new_status
    
    # Handle missing statuses
    flights_copy["flight_status"] = flights_copy["flight_status"].fillna("unknown")
    
    return flights_copy

In [23]:
def impute_delays(flights_df: pd.DataFrame, delay_column: str, scheduled_datetime_column: str, actual_datetime_column: str):

    """
    Imputing missing delay columns in several ways:
        1. If actual and scheduled datetime columns exist, use the difference to calculate delay
        2. Update unusual delay values using actual and scheduled datetime difference 
        3. If actual scheduled datetime is missing, use the median departure delays of each airline to its destination to impute missing value
           Example: All flights from Alaska Airlines heading to JFK will be grouped together
        4. If airline + destination is too sparse, use median departure delays from departure origin (i.e SFO) to impute
    Parameters: 
        flights_df (pd.DataFrame): DataFrame containing flight data 
        delay_column (str): column name for the delay column (e.g. 'dep_delay' or 'arr_delay')
        iata_column (str): column name for the iata column (e.g. 'dep_iata' or 'arr_iata')
        scheduled_datetime_column (str): column name for scheduled datetime column to compute difference
        acutal_datetime_column (str): column name for actual datetime column to compute difference
    """

    flights_copy = flights_df.copy()

    flights_copy[actual_datetime_column] = pd.to_datetime(flights_copy[actual_datetime_column]) 
    flights_copy[scheduled_datetime_column] = pd.to_datetime(flights_copy[scheduled_datetime_column]) 

    time_diff = (flights_copy[actual_datetime_column] - flights_copy[scheduled_datetime_column]).dt.total_seconds() / 60

    # Update missing (Method 1) and outlier delays (Method 2) using the time_diff if actual_departure_delay is not null
    mask_update = (flights_copy[delay_column].isna() | (flights_copy[delay_column] > 100000)) & flights_copy[actual_datetime_column].notna()
    flights_copy.loc[mask_update, delay_column] = time_diff[mask_update]

    # Impute remaining missing values using hierarchical medians
    groupby_configs = [
        (["airline", "arr_iata"], "median_delay_1"), # Method 3 
        (["dep_iata"], "median_delay_2") # Method 4
    ]
    
    for groupby_cols, median_col in groupby_configs:
        # Calculate median delays for current grouping
        median_delays = flights_copy.groupby(groupby_cols)[delay_column].median().reset_index()
        median_delays.rename(columns={delay_column: median_col}, inplace=True)

        # Merge and fill missing values
        flights_copy = pd.merge(flights_copy, median_delays, how="left", on=groupby_cols)
        mask_missing = flights_copy[delay_column].isna()
        flights_copy.loc[mask_missing, delay_column] = flights_copy.loc[mask_missing, median_col]
        flights_copy.drop(columns=median_col, inplace=True)
    
    return flights_copy


In [24]:
def agg_delay_metrics(flights_df: pd.DataFrame):

    """
    Aggregate function allowing exploration of delays by airline, route, and airport
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data 
    Returns
        DataFrame that contains the following aggregate data:
            - total flight counts
            - average departure & arrival delays
            - standard deviations for dep delays
            - min/max departure & arrival delays 
            - total delays
            - number of flights with >15 minute delays
    Note: Some standard deviation values may be missing due to groups containing only one flight, for which variability cannot be calculated.
    """
    flights_copy = flights_df.copy()

    # Calculate the total delays of each record
    flights_copy["total_delay"] = flights_copy["dep_delay"].fillna(0) + flights_copy["arr_delay"].fillna(0)

    # Mask for 15 minute delays
    flights_copy["is_delayed_15min"] = flights_copy["total_delay"] > 15

    df_agg = (
        flights_copy.groupby(["flight_date", "dep_iata", "airline", "arr_iata"])
        .agg(
            flight_count = ("flight_date", "count"),
            median_dep_delay = ("dep_delay", "median"),
            median_arr_delay = ("arr_delay", "median"),
            std_dep_delay = ("dep_delay", "std"),
            std_arr_delay = ("arr_delay", "std"), 
            min_dep_delay = ("dep_delay", "min"),
            max_dep_delay = ("dep_delay", "max"), 
            min_arr_delay = ("arr_delay", "min"),
            max_arr_delay = ("arr_delay", "max"), 
            avg_total_dep_delay = ("total_delay", "mean"),
            delayed_15min_count=('is_delayed_15min', 'sum')      
        )
        .reset_index()
    )

    return df_agg

In [25]:
def agg_flight_distribution(flights_df: pd.DataFrame):

    """
    Aggregate function to understand airport operations.
    Parameters:
        flights_df (pd.DataFrame): DataFrame containing flight data.
    Returns:
        DataFrame with:
            - flight date
            - departure airport
            - day of week (Monday - Sunday)
            - hour of day (0 - 23)
            - flight count
            - average delay (median)
    """
    flights_copy = flights_df.copy()

    # Convert date and time columns to datetime
    flights_copy["scheduled_departure_datetime_date"] = pd.to_datetime(
        flights_copy["scheduled_departure_datetime_date"], errors="coerce"
    )
    flights_copy["scheduled_departure_datetime_time"] = pd.to_datetime(
        flights_copy["scheduled_departure_datetime_time"], format="%H:%M:%S", errors="coerce"
    )

    # Extract day of week and hour only from valid datetime entries
    flights_copy["departure_day_of_week"] = flights_copy["scheduled_departure_datetime_date"].dt.day_name()
    flights_copy["hour"] = flights_copy["scheduled_departure_datetime_time"].dt.hour

    # Grouping and aggregation (NaNs will be automatically skipped in groupby)
    df_agg = (
        flights_copy.groupby(["flight_date", "dep_iata", "departure_day_of_week", "hour"])
        .agg(
            flight_count=("flight_date", "count"),
            median_delay=("dep_delay", "median")
        )
        .reset_index()
    )

    return df_agg


In [ ]:
def load_data(df, username, database_password, hostname, port, database_name, table_name, create_database=True, if_exists="append", version_tag=None):
    """
    Load the data into a MySQL database
    Parameters:
        df (pd.DataFrame): DataFrame containing aggregate or non-aggregate data
        username (str): MySQL password
        database_password (str): MySQL database password
        hostname (str): MySQL hostname or IP address
        port (str): MySQL port nummber
        database_name (str): name of the existing or non-existing database
        table_name (str): name of table stored in the database (default nanme is 'flights')
        create_database (bool): Set to 'True' to create a new database via Python, or 'False' if the database already exists in your MySQL server (default set to True)
        if_exists (str): Set to "append" or "replace" (default is "append")
        version_tag (str): Optional tag or suffix to distinguish the table (e.g., 'v1', '20250713', 'toJapan')
    Returns:
        None
    """
    from sqlalchemy import create_engine, text

    try:
        # Optional suffix to avoid overwriting previous data
        full_table_name = f"{table_name}_{version_tag}" if version_tag else table_name

        # Connection to MySQL sever (no DB yet)
        engine = create_engine(f"mysql+pymysql://{username}:{database_password}@{hostname}:{port}")

        if create_database:
            # Creates database if it doesn't exist
            with engine.connect() as conn:
                conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database_name}"))
                print(f"Database '{database_name}' is created")

        # Connect to the database
        db_engine = create_engine(f"mysql+pymysql://{username}:{database_password}@{hostname}:{port}/{database_name}")

        # Load the data into the specified table
        df.to_sql(full_table_name, con=db_engine, if_exists=if_exists, index=False)

        print(f"Data loaded into table '{table_name} in database '{database_name}'.")

    except Exception as e:
        print("Error during data loading", e)
    

In [ ]:
### Customizable parameters ###

# API 
access_key = 'Change to your API Key' # Enter your API key
dep_iata_code = ['LAX', 'JFK', 'SFO'] # Enter your IATA codes of departure airports to query
start_date = "2025-06-01" # Enter your start date in 'YYYY-MM-DD' format
end_date = "2025-06-30" # Enter your end date in 'YYYY-MM-DD' format
arr_iata_code = ['HND'] # Enter your IATA codes of arrival airports to query
airline_code_list = ["DL", "UA", "AA"] # Enter your airline codes 

# MySQL Credentials
username = "Change to your MySQL username" # Enter your username
database_password = "Change to your password" # Enter your database password
hostname = "hostname" # Enter your hostname or IP address
port = "3306" # Enter the port on which MySQL is running
database_name = "aviation_db" # Enter the name of the database 

# Optional changes to default parameters
create_database = True # Set to True to create a new database via Python (default), or False if storing in an existing db is preferred
if_exists = "replace" # Choose "append" to add new data to the existing table (default), or "replace" to overwrite the table with new data.
version_tag="v2" # Optional identifier for tables

In [32]:
## API Call
all_flights_response = get_flights_in_range(
    access_key=access_key,
    start_date=start_date,
    end_date=end_date,
    dep_code_list=dep_iata_code,
    airline_code_list=airline_code_list
)

In [ ]:
flights_df = parse_flight_response(all_flights_response)

In [36]:
# Merge cities
flights_df = merge_cities(flights_df, "airports.csv", "dep_iata", "dep_") 
flights_df = merge_cities(flights_df, "airports.csv", "arr_iata", "arr_")

# Drop duplicates
flights_df = drop_duplicates(flights_df)

# Parse datetime
flights_df = parse_datetime(flights_df, "scheduled_departure_datetime")
flights_df = parse_datetime(flights_df, "scheduled_arrival_datetime")

# Change status
flights_df = change_status(flights_df)


In [ ]:
# Ask user whether to apply imputations
impute_input = input("The next step involves imputing missing delay values. Methods include: " \
                    "1. Calculate delay from actual and scheduled datetime if both exist. " \
                    "2. Correct unusual delay values using actual vs scheduled datetime. " \
                    "3. If missing, use median departure delays for that airline to the destination. " \
                    "4. If still missing, use median departure delays from the departure airport. " \
                    "Type 'Y' to apply imputations, or 'N' to remove missing entries: "
                    )

if impute_input.lower() == "y":
    flights_df = impute_delays(flights_df, "dep_delay", "scheduled_departure_datetime", "actual_departure_datetime")
    flights_df = impute_delays(flights_df, "arr_delay", "scheduled_arrival_datetime", "actual_arrival_datetime")

In [ ]:
# Aggregate delay metrics
delay_metrics = agg_delay_metrics(flights_df)

# Aggregate flight distribution
flight_distribution = agg_flight_distribution(flights_df)

In [ ]:
flights_df = flights_df[['flight_date', 'flight_number', 'flight_status', 'airline',
                         'dep_airport', 'dep_timezone', 'dep_iata', 'dep_delay', 'scheduled_departure_datetime', 'scheduled_departure_datetime_date', 'scheduled_departure_datetime_time', 'actual_departure_datetime', 
                         'dep_latitude', 'dep_longitude', 'dep_country', 'dep_city', 'dep_state',
                         'arr_airport', 'arr_timezone', 'arr_iata', 'arr_delay', 'scheduled_arrival_datetime', 'scheduled_arrival_datetime_date', 'scheduled_arrival_datetime_time', 'actual_arrival_datetime', 
                         'arr_latitude', 'arr_longitude', 'arr_country', 'arr_city', 'arr_state'
                         ]]

In [ ]:
# Load the flights data 
df_list = [flights_df, delay_metrics, flight_distribution]
table_names = ["flights", "delays", "flight_distribution"]

for df, table_name in zip(df_list, table_names):
    load_data(
        df=df,
        username=username, 
        database_password=database_password,
        hostname=hostname,
        port=port,
        database_name=database_name,
        table_name = table_name,
        create_database=create_database,
        version_tag=version_tag
    )

Database 'aviation_db' is created
Data loaded into table 'flights in database 'aviation_db'.
Database 'aviation_db' is created
Data loaded into table 'delays in database 'aviation_db'.
Database 'aviation_db' is created
Data loaded into table 'flight_distribution in database 'aviation_db'.
